In [1]:
import os
import seaborn as sns
import matplotlib.pyplot as plt
import pickle 
import matplotlib as mpl

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('../')

from infrasim.optimise import *
from infrasim.utils import *

mpl.rcParams['ytick.direction'] = 'out'
mpl.rcParams['xtick.direction'] = 'out'
mpl.rcParams['font.family']      = 'Arial'

In [2]:
# SUBFUNCTIONS

def merge_capacity_data(results_dict):
    '''Returns a dataframe with merged capacity data based on a dictionary containing a set of results class
    '''
    # init blank df
    capacities = pd.DataFrame()
    # loop results 
    for k in results_dict.keys():
        df = results_dict[k].results_capacities
        capacities = capacities.append(df,ignore_index=True)
    return capacities


def get_relative_capacities(results_dict):
    capacities = merge_capacity_data(results_dict)
    # get bas and delta dfs
    bas   = capacities.loc[capacities.scenario.isin(['BAS'])].reset_index(drop=True)
    delta = capacities.loc[~capacities.scenario.isin(['BAS'])].reset_index(drop=True)
    # group
    bas   = bas.groupby(by=['scenario','technology','territory']).max().reset_index()
    delta = delta.groupby(by=['scenario','technology','territory']).max().reset_index()
    # map baseline capacities
    bas = bas.set_index('node')['value'].to_dict()
    delta['bas_cap'] = delta.node.map(bas)
    # compute delta
    delta['relative_change'] = delta['value'] - delta['bas_cap']
    return delta


def merge_cost_data(results_dict):
    '''Returns a dataframe with merged capacity data based on a dictionary containing a set of results class
    '''
    # init blank df
    costs = pd.DataFrame()
    # loop results 
    for k in results_dict.keys():
        df = results_dict[k].results_costs
        costs = costs.append(df,ignore_index=True)
    return costs


def export_results_for_chord(merged_results,scenario):
    '''Export a csv for chord plots
    '''
    df = merged_results[scenario].results_edge_flows.copy()

    df['from_territory'] = ''
    df['to_territory'] = ''
    df = df[df.from_id.str.contains('generation')].reset_index(drop=True)

    df.loc[df.from_id.str.contains('israel'),'from_territory'] = 'ISR'
    df.loc[df.from_id.str.contains('west_bank'),'from_territory'] = 'WBK'
    df.loc[df.from_id.str.contains('gaza'),'from_territory'] = 'GZA'
    df.loc[df.from_id.str.contains('jordan'),'from_territory'] = 'JOR'
    df.loc[df.from_id.str.contains('egypt'),'from_territory'] = 'EGY'

    df.loc[df.to_id.str.contains('israel'),'to_territory'] = 'ISR'
    df.loc[df.to_id.str.contains('west_bank'),'to_territory'] = 'WBK'
    df.loc[df.to_id.str.contains('gaza'),'to_territory'] = 'GZA'
    df.loc[df.to_id.str.contains('jordan'),'to_territory'] = 'JOR'
    df.loc[df.to_id.str.contains('egypt'),'to_territory'] = 'EGY'
    
    df = df[['from_territory','to_territory','value']]
    df = df.groupby(by=['from_territory','to_territory']).sum().reset_index()
    
    # units?
    df.value = df.value.divide(10**3)
    # save
    df.to_csv('plots/chord_data/' + scenario + '.csv',index=False)
    print('done.')
    
    
def map_territory(df,on):
    '''Map territory onto dataframe
    '''
    # add regions
    df['territory'] = np.nan
    df.loc[df[on].str.contains('jordan'),'territory']  = 'Jordan'
    df.loc[df[on].str.contains('israel'),'territory']  = 'Israel'
    df.loc[df[on].str.contains('west'),'territory']    = 'West Bank'
    df.loc[df[on].str.contains('gaza'),'territory']    = 'Gaza'
    df.loc[df[on].str.contains('super'),'territory']   = 'Shortage'
    df.loc[df[on].str.contains('egypt'),'territory']   = 'Egypt'
    return df


def map_technology(df,on):
    '''Map technology onto dataframe
    '''
    df['technology'] = np.nan
    df.loc[df.from_id.str.contains('solar'),'technology']       = 'Solar'
    df.loc[df.from_id.str.contains('coal'),'technology']        = 'Coal'
    df.loc[df.from_id.str.contains('diesel'),'technology']      = 'Diesel'
    df.loc[df.from_id.str.contains('ccgt'),'technology']        = 'Gas'
    df.loc[df.from_id.str.contains('gas'),'technology']         = 'Gas'
    df.loc[df.from_id.str.contains('shale'),'technology']       = 'Shale'
    df.loc[df.from_id.str.contains('wind'),'technology']        = 'Wind'
    df.loc[df.from_id.str.contains('storage'),'technology']     = 'Storage'
    df.loc[df.from_id.str.contains('super'),'technology']       = 'Shortage'
    return df


def map_sox_emissions(df,on):
    df['sox'] = df[on] * df['technology'].map(sox)
    return df


def map_nox_emissions(df,on):
    df['nox'] = df[on] * df['technology'].map(nox)
    return df


def map_co2_emissions(df,on):
    df['co2'] = df[on] * df['technology'].map(co2)
    return df

def map_water_use(df,on):
    df['water_use'] = df[on] * df['technology'].map(water_use)
    return df

## Load scenario results

In [ ]:
file = open('../outputs/results/model_run_results.pkl','rb')
results = pickle.load(file)
file.close()

In [ ]:
export_results_for_chord(results,scenario='BAU')
export_results_for_chord(results,scenario='NCO')
export_results_for_chord(results,scenario='EAG')
export_results_for_chord(results,scenario='COO')
#export_results_for_chord(results,scenario='UTO')

## Plot: summary of scenario analysis

This figure shows: 

    a. Total cost ($) by scenario
    b. Total capacity (GW) by scenario and technology
    c. Summary of various variables by scenario (e.g., NOx, SOx etc.)

In [ ]:
merged_flows = pd.DataFrame()
for k in results.keys():
    sn = get_source_nodes(results[k].nodes).name.to_list()
    df = results[k].results_edge_flows.loc[results[k].results_edge_flows.from_id.isin(sn)]
    df = df.drop(['to_id','commodity','timestep','hour'],axis=1)
    df = df.groupby(by=['from_id','day','month','year','scenario']).sum().reset_index()
    df = map_territory(df,on='from_id')
    df = map_technology(df,on='from_id')
    merged_flows = merged_flows.append(df,ignore_index=True)

merged_flows = map_sox_emissions(merged_flows,on='value')
merged_flows = map_nox_emissions(merged_flows,on='value')
merged_flows = map_co2_emissions(merged_flows,on='value')
merged_flows = map_water_use(merged_flows,on='value')

merged_flows = merged_flows.groupby(by=['month','year','scenario']).sum().reset_index()

# adjust units
merged_flows['co2'] = merged_flows['co2'].divide(10**6)
merged_flows['nox'] = merged_flows['nox'].divide(10**6)
merged_flows['sox'] = merged_flows['sox'].divide(10**6)

monthly_means = merged_flows.groupby(by=['year','scenario']).mean().reset_index()
monthly_means = monthly_means[['scenario','sox','nox','co2','water_use']]

# append capacities
capacities = merge_capacity_data(results)
#capacities = capacities.groupby(by=['technology','scenario']).max().reset_index()
capacities = capacities.pivot_table(index=['scenario'],
                                    columns=['technology'],
                                    values='value',
                                    aggfunc='sum')

battery = capacities['Battery'].to_dict()
monthly_means['battery'] = monthly_means.scenario.map(battery)

def normalise_column(df,columns):
    df = monthly_means.copy()
    for i in columns:
        column=i
        baseline_value = df.loc[df.scenario=='BAS',column].values[0]
        df[column] = ((df[column]-baseline_value)/baseline_value)*100
    return df

monthly_means = normalise_column(monthly_means,columns=['nox','co2','sox','water_use'])
#monthly_means = monthly_means[monthly_means.scenario != 'BAS'].reset_index(drop=True)

hls_colours = sns.color_palette("Spectral", as_cmap=True)

costs = merge_cost_data(results)
#costs = costs.groupby(by=['technology','scenario']).sum().reset_index()
costs = costs.pivot_table(index=['scenario'],
                          columns=['technology'],
                          values='totex',
                          aggfunc='sum')

costs['total'] = costs.sum(axis=1)
costs = costs.reset_index(drop=False)
costs = costs[['scenario','total']]
costs = costs.sort_values('total')
costs['total'] = costs['total'].divide(10**9)
# run function
capacities = merge_capacity_data(results)
# groupby to get max values
# capacities = capacities.groupby(by=['technology','scenario']).max().reset_index()
# pivot for bar chart
capacities = capacities.pivot_table(index=['scenario'],
                                    columns=['technology'],
                                    values='value',
                                    aggfunc='max')
# sort by ascending
capacities['total'] = capacities.sum(axis=1)
capacities = capacities.sort_values("total").drop('total',axis=1)
capacities = capacities.drop(['Coal','Gas Reservoir'],axis=1)


f = plt.figure(figsize=(10, 8))
ax1 = plt.subplot(2,2,1)
ax2 = plt.subplot(2,2,2)
ax3 = plt.subplot(2,1,2)

sns.set_style("ticks")
costs.set_index('scenario').plot.bar(ax=ax1,color='teal',rot=0,legend=False)
ax1.set_xlabel('')
ax1.set_ylabel('Total Cost [US$ billion]')
ax1.set_ylim([-30*0.03,30])
ax1.set_title('a',fontweight='bold',loc='left')

capacities.divide(1000).plot.bar(stacked=True,rot=0,ax=ax2,
                                 cmap=hls_colours,edgecolor='gray')

ax2.set_xlabel('')
ax2.set_ylabel('Capacity [GW]')
ax2.set_title('b',fontweight='bold',loc='left')
ax2.set_ylim([-40*0.03,40])

monthly_means = monthly_means.melt(id_vars='scenario',value_vars=['co2','water_use','sox','nox',])
plt.axhline(y=0,linestyle='-',color='black',linewidth=1,zorder=1)
sns.scatterplot(x='variable',y='value',hue='scenario',ax=ax3,
                data=monthly_means,s=100,palette='Spectral',edgecolor='darkgray')

ax3.set_xlabel('')
ax3.set_ylabel('Change relative to BAS scenario (%)')
ax3.set_title('c',fontweight='bold',loc='left')
ax3.set_ylim([-35,10])
ax3.set_xticklabels(['CO$_2$\nemissions','Water\nwithdrawals','SO$_x$\n emissions','NO$_x$\n emissions'])

f.savefig('../outputs/figures/scenario_analysis.pdf',bbox_inches='tight')

## Plot: storage curves

This figure shows: 

    a. Monthly supply by technology
    b. Battery charge and discharge rates

In [ ]:
df = results['COO'].results_edge_flows.copy()
df = df.groupby(by=['from_id','month']).sum()['value'].reset_index()
df = df[~df.from_id.str.contains('generation')]
df = df[~df.from_id.str.contains('storage')]
df = df[df.value > 0].reset_index(drop=True)

# adjust names
df.from_id = df.from_id.str.replace('_',' ')
df.from_id = df.from_id.str.replace('ccgt','Gas')
df.from_id = df.from_id.str.replace('natural gas','Gas')
df.from_id = df.from_id.str.title()

df.value = df.value.divide(10**6)
df = df.pivot_table(index=['month'],columns=['from_id'],values='value',aggfunc='sum',)

f,ax = plt.subplots(figsize=(10,8),nrows=2,ncols=1)

df.plot.bar(stacked=True,cmap='Spectral',edgecolor='darkgray',ax=ax[0],rot=0)

ax[0].set_ylim([-14*0.03,14])
ax[0].set_ylabel('Supply (GWh)')
ax[0].set_xlabel('')
ax[0].set_xticklabels(['J','F','M','A','M','J','J','A','S','O','N','D'])
ax[0].set_title('a',fontweight='bold',loc='left')



df = results['COO'].results_edge_flows.copy()
#df = df.groupby(by=['from_id','month']).sum()['value'].reset_index()
df1 = df[df.to_id.str.contains('battery')]
df2 = df[df.from_id.str.contains('battery')]
df2.value = -df2.value
df = df1.append(df2,ignore_index=True)

df['node'] = ''
df['mode'] = ''
df.loc[df.from_id.str.contains('battery'),'node'] = df.loc[df.from_id.str.contains('battery'),'from_id']
df.loc[df.to_id.str.contains('battery'),'node'] = df.loc[df.to_id.str.contains('battery'),'to_id']
df.loc[df.value <= 0, 'mode'] = 'discharge'
df.loc[df.value > 0, 'mode'] = 'charge'
df = df[df.value != 0].reset_index(drop=True)
df.value = df.value.divide(10**3)
# rename
df.node = df.node.str.replace('battery_storage','')
df.node = df.node.str.replace('_','')
df.node = df.node.str.replace('israel','ISR')
df.node = df.node.str.replace('jordan','JOR')
df.node = df.node.str.replace('westbank','WBK')
df.node = df.node.str.replace('gaza','GZA')
df = df[['node','mode','hour','day','month','value']]
df = df.groupby(by=['node','mode','month']).mean().reset_index()

df.pivot_table(index='month',columns=['node','mode'],values='value').plot.bar(stacked=True,
                                                                              cmap='Set3',
                                                                              edgecolor='darkgray',
                                                                              rot=0,
                                                                              ax=ax[1])

ax[1].set_ylim([-7,6])
ax[1].set_ylabel('Battery charge/discharge (MWh)')
ax[1].set_xlabel('')
ax[1].set_xticklabels(['J','F','M','A','M','J','J','A','S','O','N','D'])
ax[1].set_title('b',fontweight='bold',loc='left')

f.savefig('../outputs/figures/coo_supply_curves.pdf',bbox_inches='tight')

## Plot: capacities by scenario for each region

This figure shows the capacity of each technology implemented under each scenario.

In [ ]:
caps = merge_capacity_data(results)
caps = caps.groupby(by=['node','scenario']).max().reset_index()
caps = caps.loc[caps.value>0].reset_index(drop=True)
caps.value = caps.value.divide(1000)

titles = {'Gaza':'GZA','Israel':'ISR','West Bank':'WBK','Jordan':'JOR',}
ylims = [ [-0.3*0.03,0.3],[-16*0.03,16],[-14*0.03,14],[-1.2*0.03,1.2] ]

f,axs = plt.subplots(2,2, figsize=(10,8))
ax = axs.ravel()
for i in range(len(caps.territory.unique())):
    idx_caps = caps.loc[caps.territory == caps.territory.unique()[i]].reset_index(drop=True)
    sns.barplot(x='scenario',y='value',hue='technology',data=idx_caps,ax=ax[i],errwidth=0)
    ax[i].set_xlabel('')
    ax[i].set_ylabel('Capacity (GW)')
    ax[i].set_title(titles[caps.territory.unique()[i]],loc='left',fontweight='bold')
    ax[i].set_ylim(ylims[i])
    
f.savefig('../outputs/figures/supp_capacity_by_scenario.pdf',bbox_inches='tight')

## Plot: operational curves

This figure shows how many days a given region is dependant upon another state for its electricity. 

In [ ]:
df = results['COO'].results_edge_flows.copy()
# remove gas storages
df = df[~df.from_id.str.contains('reservoir')]
# loop through storage nodes
for s in df[df.from_id.str.contains('storage')].from_id.unique():
    inflow = df.loc[df.to_id == s].reset_index(drop=True).groupby(\
                by=['timestep','hour','day','month','year']).sum().reset_index()
    
    outflow = df.loc[df.from_id == s].reset_index(drop=True).groupby(\
                by=['timestep','hour','day','month','year']).sum().reset_index()

    charge = outflow.copy()
    charge['value'] = outflow['value'] - inflow['value']
    charge['from_id'] = s
    charge['scenario'] = df.scenario.unique()[0]
    charge['commodity'] = df.commodity.unique()[0]
    df = df[~df.from_id.isin([s])].reset_index(drop=True)
    df = df.append(charge,ignore_index=True)

df = df.drop('to_id',axis=1)

results['COO'].results_edge_flows = df.copy()

In [ ]:
results['COO'].results_edge_flows

In [ ]:
f,ax=plt.subplots(nrows=2,ncols=2,figsize=(10,6),sharex=True)

results['COO'].plot_hourly_profile(day=1,month=6,year=2030,territory='Israel',
                                   cmap='Set2',linewidth=1,ax=ax[0,0])

results['COO'].plot_hourly_profile(day=1,month=6,year=2030,territory='Jordan',
                                   cmap='Set2',linewidth=1,ax=ax[0,1])

results['COO'].plot_hourly_profile(day=1,month=6,year=2030,territory='West Bank',
                                   cmap='Set2',linewidth=1,ax=ax[1,0])

results['COO'].plot_hourly_profile(day=1,month=6,year=2030,territory='Gaza',
                                   cmap='Set2',linewidth=1,ax=ax[1,1])

ax[0,0].set_ylim([0,6000])
ax[0,1].set_ylim([-2000,16000])
ax[1,0].set_ylim([0,400])
ax[1,1].set_ylim([0,70])

ax[0,0].set_xlim([1,24])
ax[0,1].set_xlim([1,24])
ax[1,0].set_xlim([1,24])
ax[1,1].set_xlim([1,24])

ax[0,0].set_ylabel('Supply (kWh)')
ax[1,0].set_ylabel('Supply (kWh)')
ax[1,0].set_xlabel('Hour')
ax[1,1].set_xlabel('Hour')

ax[0,0].set_title('a',loc='left',fontweight='bold',fontsize=14)
ax[0,1].set_title('b',loc='left',fontweight='bold',fontsize=14)
ax[1,0].set_title('c',loc='left',fontweight='bold',fontsize=14)
ax[1,1].set_title('d',loc='left',fontweight='bold',fontsize=14)

## Visualise flows
This figure visualises flows between each territory

###### NOTE TO SELF:
Here I'm trying to generate a set of sankey plots in a loop (currently testing just for NCO) but can't get the plotly figure to save... getting the error:

> The colour should be an integer rather than string... maybe?
> When colour arguments are commented out, I get the following error: 01 The kaleido executable is required by the kaleido Python library, but it was not included

In [ ]:
# import os
# import seaborn as sns
# import matplotlib.pyplot as plt

# import warnings
# warnings.filterwarnings('ignore')

# import sys
# sys.path.append('../')

# from infrasim.optimise import *
# from infrasim.utils import *

# # SUBFUNCTIONS
# def merge_capacity_data(results_dict):
#     '''Returns a dataframe with merged capacity data based on a dictionary containing a set of results class
#     '''
#     # init blank df
#     capacities = pd.DataFrame()
#     # loop results 
#     for k in results_dict.keys():
#         df = results_dict[k].results_capacities
#         capacities = capacities.append(df,ignore_index=True)
#     return capacities


# def get_relative_capacities(results_dict):
#     capacities = merge_capacity_data(results_dict)
#     # get bas and delta dfs
#     bas   = capacities.loc[capacities.scenario.isin(['BAS'])].reset_index(drop=True)
#     delta = capacities.loc[~capacities.scenario.isin(['BAS'])].reset_index(drop=True)
#     # group
#     bas   = bas.groupby(by=['scenario','technology','territory']).max().reset_index()
#     delta = delta.groupby(by=['scenario','technology','territory']).max().reset_index()
#     # map baseline capacities
#     bas = bas.set_index('node')['value'].to_dict()
#     delta['bas_cap'] = delta.node.map(bas)
#     # compute delta
#     delta['relative_change'] = delta['value'] - delta['bas_cap']
#     return delta

# #File paths
# nodes = '../data/nextra/spatial/network/nodes.shp'
# edges = '../data/nextra/spatial/network/edges.shp'
# flows = '../data/nextra/nodal_flows/processed_flows_2030.csv'

# # Params
# timesteps=24#7000
# super_source=False
# pprint=True
# save_figures=True

# infrasim_init_directories()

# scenarios = {
#              'NCO' : True,
#             }

# results = {}
# for s in scenarios:
        
#     model_run = nextra(nodes,edges,flows,
#                        scenario=s,
#                        energy_objective=scenarios[s],
#                        timesteps=timesteps,
#                        #super_source=True,
#                        #super_sink=True,
#                        #res_factor=99,
#                        #model_name='meow',
#                       )

#     model_run.build()
#     model_run.run(pprint=False)
#     try:
#         model_results = model_run.get_results()
#         # add scenarios to results
#         if s == 'BAU' and scenarios[s] == False:
#             s = 'BAS'
#         model_results.results_capacities['scenario']       = s
#         model_results.results_storages['scenario']         = s
#         model_results.results_edge_flows['scenario']       = s
#         model_results.results_capacity_change['scenario']  = s
#         model_results.results_costs['scenario']            = s
#         # append results
#         results[s] = model_results
#         print('> Completed: ' + s)
#     except:
#         print('> FAILED! ' + s)
#         os.system('say "Scenario failed"')

# #os.system('say "Code block finished"')

# print('> Done.')

# results['NCO'].plot_flows_sankey(height=500,title='No Cooperation')